<a href="https://colab.research.google.com/github/lautitorraca/7506R-2C2023-GRUPO24/blob/main/TP2/7506R_TP2_GRUPO24_RED_NEURONAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

𝔾𝕣𝕦𝕡𝕠 𝟚𝟜 (Merequetengue👍)

➢ Torraca Lautaro - 108813

➢ Negrotti Gianluca - 108184

➢ Marco Tosi - 107237

### > Imports

In [41]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras.metrics import BinaryAccuracy
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow import keras
from sklearn.preprocessing import StandardScaler

# Optimización de hiperparámetros
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, RepeatedKFold

# Manejo de modelos
from pickle import dump
from pickle import load

### > Lectura de los datasets

In [42]:
url_train = '/content/drive/MyDrive/Colab Notebooks/train_limpio.csv'
url_test = '/content/test_limpio.csv'

df_train = pd.read_csv(url_train)
df_test = pd.read_csv(url_test)

train = df_train.copy()
test = df_test.copy()

- Division y preprocesamiento de los datos



In [29]:
target = 'sentimiento'
features = 'review_es_clean'

In [30]:
X_train, X_test, y_train, y_test = train_test_split(train[features],
                                                    train[target],
                                                    test_size=0.3,
                                                    random_state=42,
                                                    stratify=train[target],
                                                    shuffle=True)



### Red neuronal

- Tokenizacion



In [21]:
# Convertir las críticas de texto a secuencias numéricas
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
word_index = tokenizer.word_index

# Ajustar el tamaño del vocabulario en la capa Embedding
vocab_size = min(len(word_index) + 1, 20000)

# Limitar la longitud de las secuencias a un tamaño fijo
max_sequence_length = 200
X_train_padded = pad_sequences(sequences, maxlen=max_sequence_length)

# Convertir las etiquetas a valores numéricos
labels = np.array(y_train.replace({"negativo": 0, "positivo": 1}))


- Creacion del modelo de RNN

In [ ]:
# Crear el modelo de RNN con capas GRU
model_opt = Sequential()
model_opt.add(Embedding(vocab_size, 100, input_length=max_sequence_length))
model_opt.add(GRU(64, dropout=0.2))  # Considera quitar recurrent_dropout si afecta el rendimiento en GPU
model_opt.add(Dense(1, activation='sigmoid'))

# Configurar el optimizador
optimizer = optimizers.Adam()

# Compilar el modelo
model_opt.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[BinaryAccuracy(name='accuracy')])

# Cantidad de épocas de entrenamiento
cant_epochs = 8





- Creacion de funcion para optimizar f1

In [ ]:
def f1_score(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2 * p * r / (p + r + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)


- Entrenamiento del modelo

In [ ]:
# Compilar el modelo
model_opt.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[f1_score])

# Cantidad de épocas de entrenamiento
cant_epochs = 8

# Configurar el callback de Early Stopping para monitorear el f1_score
early_stopping = EarlyStopping(monitor='val_f1_score',
                               patience=4,
                               mode='max',
                               restore_best_weights=True,
                               min_delta=0.0001)

# Entrenar el modelo
history_model_rn = model_opt.fit(X_train_padded,
                                 labels,
                                 epochs=cant_epochs,
                                 batch_size=32,
                                 validation_split=0.2,
                                 verbose=1,
                                 callbacks=[early_stopping])


Epoch 1/8
875/875 [==============================] - 137s 154ms/step - loss: 0.4299 - f1_score: 0.7950 - val_loss: 0.3410 - val_f1_score: 0.8581
Epoch 2/8
875/875 [==============================] - 133s 152ms/step - loss: 0.2352 - f1_score: 0.9077 - val_loss: 0.3551 - val_f1_score: 0.8423
Epoch 3/8
875/875 [==============================] - 135s 154ms/step - loss: 0.1446 - f1_score: 0.9481 - val_loss: 0.3981 - val_f1_score: 0.8521
Epoch 4/8
875/875 [==============================] - 134s 153ms/step - loss: 0.0845 - f1_score: 0.9693 - val_loss: 0.4784 - val_f1_score: 0.8093
Epoch 5/8
875/875 [==============================] - 129s 148ms/step - loss: 0.0511 - f1_score: 0.9823 - val_loss: 0.6152 - val_f1_score: 0.8417


- Prediccion y muestra de la matriz

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np


# Convertir las críticas de texto de prueba a secuencias numéricas
sequences_test = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(sequences_test, maxlen=max_sequence_length)

# Realizar predicciones
predictions = model_opt.predict(X_test_padded)

# Convertir las predicciones a etiquetas binarias
predicted_labels = [1 if p > 0.5 else 0 for p in np.ravel(predictions)]

# Convertir y_test a valores numéricos si es necesario
y_test_numeric = np.array(y_test.replace({"negativo": 0, "positivo": 1}))

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_numeric, predicted_labels)
print("Matriz de Confusión:")
print(conf_matrix)


469/469 [==============================] - 16s 33ms/step
Matriz de Confusión:
[[6343 1157]
 [ 950 6550]]


- Guardado del modelo en H5 (similar a pickle)

In [ ]:
model_opt.save('red_neuronal.h5')  # Guarda el modelo en formato H5
from google.colab import files
files.download('red_neuronal.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

- Prediccion con el dataset Test

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convertir las revisiones de texto a secuencias numéricas
test_sequences = tokenizer.texts_to_sequences(test['review_es_clean'])


max_sequence_length = 200
X_test_tokenizer = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Realizar predicciones
predictions = model_opt.predict(X_test_tokenizer)
predicted_sentiment = ['positivo' if pred > 0.5 else 'negativo' for pred in predictions]

# Agregar las predicciones al DataFrame de prueba
test['predicted_sentiment'] = predicted_sentiment

# Seleccionar solo las columnas ID y predicted_sentiment
output = test[['ID', 'predicted_sentiment']]

# Guardar las predicciones en un nuevo archivo CSV
output.to_csv('test_predictions.csv', index=False)


269/269 [==============================] - 10s 36ms/step


- Descarga de la prediccion

In [ ]:
# Descargar el archivo (útil en Jupyter Notebook o Google Colab)
try:
    from google.colab import files
    files.download('test_predictions.csv')
except ImportError:
    print("La función de descarga solo funciona en entornos como Google Colab.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### >  Segundo modelo de Red Neuronal

In [60]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences




# Codificación de las etiquetas
label_encoder = LabelEncoder()
train['sentimiento_encoded'] = label_encoder.fit_transform(train['sentimiento'])

# Vectorización de las reseñas
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train['review_es_clean'])

X = tokenizer.texts_to_sequences(train['review_es_clean'])
X = pad_sequences(X, maxlen=200) # Ajusta este valor según la longitud de tus reseñas

y = train['sentimiento_encoded']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [66]:
# Parámetros del modelo
cant_clases = len(np.unique(y_train))
d_in = X_train.shape[1]

modelo_1 = keras.Sequential([
    keras.layers.Dense(32, input_shape=(d_in,), activation='relu', kernel_initializer='uniform'), # Capa de entrada
    keras.layers.Dense(32, activation='relu'),

    keras.layers.Dropout(0.6),

    keras.layers.Dense(32, input_shape=(d_in,), activation='relu', kernel_initializer='uniform'),
    keras.layers.Dense(32, activation='relu'),

    keras.layers.Dropout(0.6),
    keras.layers.Dense(cant_clases, activation='sigmoid') # Capa de salida
])

modelo_1.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])


In [67]:
history = modelo_1.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


Epoch 1/5
1000/1000 [==============================] - 6s 5ms/step - loss: 1.3807 - accuracy: 0.4990 - val_loss: 0.6928 - val_accuracy: 0.5000
Epoch 2/5
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6950 - accuracy: 0.4985 - val_loss: 0.6927 - val_accuracy: 0.4991
Epoch 3/5
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6941 - accuracy: 0.5059 - val_loss: 0.6933 - val_accuracy: 0.4891
Epoch 4/5
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6934 - accuracy: 0.4982 - val_loss: 0.6932 - val_accuracy: 0.5008
Epoch 5/5
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6937 - accuracy: 0.4987 - val_loss: 0.6941 - val_accuracy: 0.5058


- Evaluacion y prediccion del modelo

In [49]:
# Evaluación del modelo
test_loss, test_acc = modelo_1.evaluate(X_test, y_test)

print('Test Accuracy:', test_acc)

# Realizar predicciones
predictions = modelo_1.predict(X_test)


313/313 [==============================] - 1s 2ms/step - loss: 0.6932 - accuracy: 0.4961
Test Accuracy: 0.4961000084877014
313/313 [==============================] - 1s 1ms/step


- Guardado del archivo con pickle

In [50]:
import pickle

# Guardar el modelo
with open('modelo_1.pkl', 'wb') as file:
    pickle.dump(modelo_1, file)


- Prediccion con el dataset de test

In [53]:

X_nuevo = tokenizer.texts_to_sequences(test['review_es_clean'])
X_nuevo = pad_sequences(X_nuevo, maxlen=200)  # Usa la misma longitud máxima que antes


In [54]:
nuevas_predicciones = modelo_1.predict(X_nuevo)


269/269 [==============================] - 0s 2ms/step


- Preparacion del csv para bajar

In [55]:


# Convertir las salidas del modelo en índices de clase
indices_predicciones = np.argmax(nuevas_predicciones, axis=1)


mapeo_etiquetas = {0: 'Negativo', 1: 'Positivo'}
etiquetas_predicciones = [mapeo_etiquetas[i] for i in indices_predicciones]


- Bajado del archivo de prediccion

In [59]:
# Supongamos que 'nuevo_dataset' tiene una columna 'ID'
nuevo_dataset_predicciones = pd.DataFrame({
    'ID': test['ID'],
    'sentimiento': etiquetas_predicciones
})

# Guardar en un archivo CSV
nuevo_dataset_predicciones.to_csv('predicciones_red2.csv', index=False)
try:
    from google.colab import files
    files.download('predicciones_red2.csv')
except ImportError:
    print("La función de descarga solo funciona en entornos como Google Colab.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>